## INFOH515 Pyspark code
## Author: Gianluca Bontempi
## Pyspark implementation of K-means

In [1]:
import numpy as np
import pwd
import getpass
import os
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, sum
userName = getpass.getuser()
appName = pwd.getpwuid( os.getuid() )[ 0 ]


# create an instance of SparkSession
spark=SparkSession.builder.appName('s.com').getOrCreate()
sc=spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 17:11:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Import dataset

In [2]:
dataset = sc.textFile("Dclust.txt").map(lambda x : x.split(",")).map(lambda x : [float(i) for i in x])

In [3]:
n=len(dataset.take(1)[0])-1
print('N=',dataset.count(), 'n=', n)

[Stage 0:>                                                          (0 + 1) / 1]

N= 258 n= 4


#### Mapstep function: returns the closest cluster for a given observation

In [30]:
def mapstep(x,CL):
    M=CL.value.shape[0]
    d=np.zeros(M)
    for m in np.arange(M):
        d[m]=np.sum(abs(x[1:]-CL.value[m,]))
    
    return(np.argmin(d))   

### K-means implementation

In [31]:
np.random.seed(0)
K=3 ## number of clusters

Bcl=np.random.randn(K,n) ## random initialization
broadcastClusters = sc.broadcast(Bcl) ## broadcast cluster position
broadcastClusters.value

for it in range(20):
    d2=dataset.map(lambda x: (mapstep(x,broadcastClusters),(np.array(x),1)))  ## mapstep
    newC=d2.reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1])).map(lambda x: x[1][0][1:]/x[1][1]).take(K)
    M=len(newC)
    Bcl=2*np.random.randn(K,n)
    for k in range(K):
        Bcl[k]=newC[k]
    broadcastClusters = sc.broadcast(Bcl)  ## update braodcast value
    print('it=',it, "K=",K, "clusters \n position=",Bcl)

it= 0 K= 3 clusters 
 position= [[-0.11193304  0.2937566  -0.49111279  0.18846489]
 [ 1.60407815  0.75034265  0.30560988  1.32770685]
 [ 0.70036898 -0.78355826  0.01855897 -0.41265748]]
it= 1 K= 3 clusters 
 position= [[-0.20207675  0.40478273 -0.588075    0.07488165]
 [ 0.46783018 -0.84023923  0.03481103 -0.32099468]
 [ 1.28587506  0.64414552 -0.05656896  1.03275604]]
it= 2 K= 3 clusters 
 position= [[ 1.09506323  0.60577873 -0.08841171  0.85327946]
 [-0.21141209  0.44369733 -0.7012311  -0.02562208]
 [ 0.36021018 -0.87168795  0.0929273  -0.24424991]]
it= 3 K= 3 clusters 
 position= [[ 0.86756923  0.52206512  0.08239691  0.82293908]
 [ 0.31177611 -0.90357751  0.12830594 -0.26249004]
 [-0.21021995  0.39303251 -0.88835434 -0.16355409]]
it= 4 K= 3 clusters 
 position= [[ 0.76194065  0.54374624  0.11537969  0.76496201]
 [-0.20220866  0.29949696 -1.0088327  -0.24713719]
 [ 0.34433353 -0.92766782  0.24630479 -0.20481259]]
it= 5 K= 3 clusters 
 position= [[ 0.74889729  0.55726331  0.13033832 